<a href="https://colab.research.google.com/github/mumairhassan/AGI-P/blob/main/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 37.7 MB/s 
     |████████████████████████████████| 1.2 MB 50.8 MB/s 
     |████████████████████████████████| 6.6 MB 57.6 MB/s 
     |████████████████████████████████| 86 kB 7.5 MB/s 
     |████████████████████████████████| 596 kB 79.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"]="false"

class CFG:
    wandb=False

    debug=False
    apex=False
    print_freq=100
    num_workers=1
    model="microsoft/mdeberta-v3-base"
    #model="xlm-roberta-base"
    #model="bert-base-multilingual-uncased"
    #model="Jatin-WIAI/punjabi_relevance_clf"
    #model='bert-large-uncased-whole-word-masking'
    #model="markussagen/xlm-roberta-longformer-base-4096"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=10
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=4
    fc_dropout=0.2
    max_len=512
    weight_decay=0.00
    gradient_accumulation_steps=2
    max_grad_norm=10000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    filtering=True
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

def prepare_input(cfg, src):
    inputs = cfg.tokenizer(src,
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",truncation=True,
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs



class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.src = df['Text'].values

        self.j = df[CFG.target].values


    def __len__(self):
        return len(self.src)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg,
                               self.src[item])
        label = torch.tensor(self.j[item], dtype=torch.long)
        return inputs, label


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data_dir = "/content/drive/MyDrive/PunjabGenderIdentification/"
df = pd.read_excel(f"{data_dir}Gender_punjabi.xlsx")

df["Text"].apply(lambda x: len(x)).describe()



count    1944.000000
mean     1273.345679
std        74.012086
min      1066.000000
25%      1224.000000
50%      1267.000000
75%      1312.000000
max      1523.000000
Name: Text, dtype: float64

In [ ]:
from transformers import AutoTokenizer, AutoModel,AutoConfig
from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast
OUTPUT_DIR = "/content/drive/MyDrive/PunjabGenderIdentification/"
#tokenizer = AutoTokenizer.from_pretrained(CFG.model,do_lower_case=True, use_fast=False)
#tokenizer = AutoTokenizer.from_pretrained(CFG.model)
if "deberta-v" in CFG.model:
    tokenizer = DebertaV2TokenizerFast.from_pretrained(CFG.model)
else:
    tokenizer = AutoTokenizer.from_pretrained(CFG.model)
#tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
#CFG.tokenizer = tokenizer
#tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer
lengths = []
train = df
for src in train["Text"].values:
  length = len(tokenizer(src)['input_ids'])
  lengths.append(length)
CFG.max_len = max(lengths)
print (f"max sequence length: {CFG.max_len}")

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


max sequence length: 865


In [ ]:
class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config, numlabels):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        classifier_dropout = 0.2
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, numlabels)

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = ClassificationHead(config,1)

        self._init_weights(self.fc)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        batch_size = outputs[0].size(0)
        last_hidden_states = outputs[0]#[:,0]#.view(batch_size,-1)
        #last_hidden_states = outputs[0].view(batch_size,-1)
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        #output = self.fc(self.fc_dropout(feature))
        output = self.fc(feature)
        return output

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

def train_fn_ready_made(fold, train_loader, model, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            classified_outputs = model(**inputs,labels=labels,return_dict=True)
        #loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))


        loss = classified_outputs.loss
        loss = torch.masked_select(loss, inputs["attention_mask"]==1).mean()
        #loss = loss.mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader),
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn_ready_made(valid_loader, model, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            classified_outputs = model(**inputs,labels=labels,return_dict=True)
        y_preds = classified_outputs.logits
        loss = classified_outputs.loss
        #loss = criterion(y_preds.view(-1, 1), labels.view(-1,1))
        #loss = torch.masked_select(loss, inputs["attention_mask"]==1).mean()
        loss = loss.mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        #preds.append(y_preds.sigmoid().to('cpu').numpy())
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
        #loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))

        loss = criterion(y_preds.view(-1, 1), labels.view(-1,1))
        #loss = torch.masked_select(loss, inputs["attention_mask"]==1).mean()
        loss = loss.mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader),
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)

        loss = criterion(y_preds.view(-1, 1), labels.view(-1,1))
        #loss = torch.masked_select(loss, inputs["attention_mask"]==1).mean()
        loss = loss.mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions


In [ ]:
import sklearn
from transformers import AutoModelForSequenceClassification
def train_loop(folds, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)


    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)

    valid_labels = valid_folds[CFG.target]

    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    #model = CustomModel(CFG, config_path=None, pretrained=True)
    model = AutoModelForSequenceClassification.from_pretrained(CFG.model,num_labels=2,problem_type = "single_label_classification")
    #model.config.num_labels = 1
    #model.config.problem_type = "single_label_classification"
    torch.save(model.config, OUTPUT_DIR+f'{CFG.model.replace("/","_")}_config.pth')
    model.to(device)

    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        if "deberta" in CFG.model:
            optimizer_parameters = [
                {'params': [p for n, p in model.deberta.named_parameters() if not any(nd in n for nd in no_decay)],
                 'lr': encoder_lr, 'weight_decay': weight_decay},
                {'params': [p for n, p in model.deberta.named_parameters() if any(nd in n for nd in no_decay)],
                 'lr': encoder_lr, 'weight_decay': 0.0},
                {'params': [p for n, p in model.named_parameters() if "deberta" not in n],
                 'lr': decoder_lr, 'weight_decay': 0.0}
            ]
        elif "roberta" in CFG.model:
            optimizer_parameters = [
                {'params': [p for n, p in model.roberta.named_parameters() if not any(nd in n for nd in no_decay)],
                 'lr': encoder_lr, 'weight_decay': weight_decay},
                {'params': [p for n, p in model.roberta.named_parameters() if any(nd in n for nd in no_decay)],
                 'lr': encoder_lr, 'weight_decay': 0.0},
                {'params': [p for n, p in model.named_parameters() if "roberta" not in n],
                 'lr': decoder_lr, 'weight_decay': 0.0}
            ]
        elif "bert" in CFG.model:
            optimizer_parameters = [
                {'params': [p for n, p in model.bert.named_parameters() if not any(nd in n for nd in no_decay)],
                 'lr': encoder_lr, 'weight_decay': weight_decay},
                {'params': [p for n, p in model.bert.named_parameters() if any(nd in n for nd in no_decay)],
                 'lr': encoder_lr, 'weight_decay': 0.0},
                {'params': [p for n, p in model.named_parameters() if "bert" not in n],
                 'lr': decoder_lr, 'weight_decay': 0.0}
            ]

        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr,
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)

    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler

    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    #criterion = nn.BCEWithLogitsLoss(reduction="none")
    #criterion = nn.MSELoss(reduction="none")

    best_score = 0.

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        #avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)
        avg_loss = train_fn_ready_made(fold, train_loader, model, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn_ready_made(valid_loader, model, device)
        #predictions = predictions.reshape((len(valid_folds), CFG.max_len))

        # scoring

        #preds = np.where(predictions>0.5,1,0)
        preds = np.argmax(predictions,axis=1)

        score = sklearn.metrics.accuracy_score(valid_labels, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1,
                       f"[fold{fold}] avg_train_loss": avg_loss,
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})

        #if best_score < score:
        if True:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': preds},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_{CFG.target}_fold{fold}_best.pth")

    preds = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_{CFG.target}_fold{fold}_best.pth",
                             map_location=torch.device('cpu'))['predictions']
    valid_folds["predictions"] = preds

    torch.cuda.empty_cache()
    gc.collect()

    return valid_folds

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/PunjabGenderIdentification/finetuned/"
import os
import random
import numpy as np
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=42)

In [ ]:
from sklearn.model_selection import GroupKFold,KFold
df["Gender_bin"] = df["Gender"].apply(lambda x: 1 if x=="Female" else 0)
if CFG.filtering:
    df = df.drop_duplicates(subset=["Gender","Text"])
    df["Text_len"] = df["Text"].apply(lambda x: len(x))
    df = df[df["Text_len"]<4000]
    df = df[df["Text_len"]>170]
df = df.sample(frac=1.)
train = df.reset_index()
#train.columns = ["Text","Gender"]
Fold = KFold(n_splits=CFG.n_fold)
#groups = train['pn_num'].values
for n, (train_index, val_index) in enumerate(Fold.split(train)):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
#train["quality_bin"] = train["quality"].apply(lambda x: 0 if x=="bad" else 1)
display(train.groupby('fold').size())

fold
0    389
1    389
2    389
3    389
4    388
dtype: int64

In [ ]:
df

,Unnamed: 0,Text,Gender,Gender_bin,Text_len
1610,627,"ਨਵੀਂ ਦਿੱਲੀ, ਐਜੂਕੇਸ਼ਨ ਡੈਸਕ : 2021: ਨੀਟ ਯੂਜੀ ਕਾਊ...",Female,1,1377
1402,459,: ਦਸੰਬਰ ਦੇ ਪਹਿਲੇ ਦਿਨ ਹੀ ਪੈਟਰੋਲੀਅਮ ਕੰਪਨੀਆਂ ਨੇ ਆ...,Female,1,1255
1718,884,"ਜੇਐੱਨਐੱਨ, ਨਵੀਂ ਦਿੱਲੀ : ਕਈ ਵਾਰ ਤੁਹਾਡੇ ਫੋਨ ’ਤੇ ਕ...",Female,1,1263
1054,302,ਕਿਹਾ ਜਾਂਦਾ ਹੈ ਕਿ ਉਡਾਣ ਖੰਭਾਂ ਨਾਲ ਨਹੀਂ ਹਿੰਮਤ ਨਾਲ...,Female,1,1283
307,1489,"ਨਵੀਂ ਦਿੱਲੀ, ਬਿਜ਼ਨੈੱਸ ਡੈਸਕ : ਸੋਨੇ ਦੀ ਕੀਮਤ ਫਿਰ ਚ...",Male,0,1270
...,...,...,...,...,...
1130,500,"ਜੇਐੱਨਐੱਨ, ਨਵੀਂ ਦਿੱਲੀ। ਬਾਬਾ ਦੀਪ ਸਿੰਘ ਦਾ ਜਨਮ 27 ...",Female,1,1202
1294,563,"ਆਈਏਐੱਨਐੱਸ, ਨਵੀਂ ਦਿੱਲੀ : ਭਾਰਤੀ ਰੇਲਵੇ ਨੇ ਰਾਜਧਾਨੀ...",Female,1,1506
860,1937,ਹਰੇਕ ਸਮਾਜ ਵਿਚ ਵਿਸਾਖੀ ਦਾ ਮਹੀਨਾ ਕਈ ਤਰ੍ਹਾਂ ਮਨਾਇਆ ...,Male,0,1130
1459,896,ਸੋਰਠਿ ਮਹਲਾ ੧ ॥ ਜਿਨ੍ਹ੍ਹੀ ਸਤਿਗੁਰੁ ਸੇਵਿਆ ਪਿਆਰੇ ਤਿ...,Female,1,1229


In [ ]:
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
import time
import math
import gc
import transformers
transformers.logging.set_verbosity_error()
if __name__ == '__main__':

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    CFG.target = "Gender_bin"
    CFG.max_len = 768
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(0,CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                #get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        #get_result(oof_df)
        oof_df.to_pickle(OUTPUT_DIR+f'{CFG.model.replace("/","_")}_{CFG.target}_oof_df.pkl')

    if CFG.wandb:
        wandb.finish()

========== fold: 2 training ==========


Downloading:   0%|          | 0.00/534M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Epoch: [1][0/388] Elapsed 0m 1s (remain 7m 0s) Loss: 0.3335(0.3335) Grad: 1.7225  LR: 0.00002000  
Epoch: [1][100/388] Elapsed 1m 49s (remain 5m 10s) Loss: 0.3402(0.3407) Grad: 7.0974  LR: 0.00001999  
Epoch: [1][200/388] Elapsed 3m 41s (remain 3m 26s) Loss: 0.3071(0.2957) Grad: 6.5341  LR: 0.00001997  
Epoch: [1][300/388] Elapsed 5m 33s (remain 1m 36s) Loss: 0.0683(0.2656) Grad: 1.8941  LR: 0.00001993  
Epoch: [1][387/388] Elapsed 7m 11s (remain 0m 0s) Loss: 0.2896(0.2584) Grad: 10.0500  LR: 0.00001988  
EVAL: [0/98] Elapsed 0m 0s (remain 0m 47s) Loss: 0.0669(0.0669) 


Epoch 1 - avg_train_loss: 0.2584  avg_val_loss: 0.1966  time: 471s
Epoch 1 - Score: 0.8252
Epoch 1 - Save Best Score: 0.8252 Model


...

Epoch: [5][0/388] Elapsed 0m 0s (remain 6m 11s) Loss: 0.0399(0.0399) Grad: 5.3913  LR: 0.00001810  
Epoch: [5][100/388] Elapsed 1m 53s (remain 5m 21s) Loss: 0.0284(0.1311) Grad: 0.7002  LR: 0.00001785  
Epoch: [5][200/388] Elapsed 3m 45s (remain 3m 29s) Loss: 0.1997(0.1432) Grad: 7.2845  LR: 0.00001760  
Epoch: [5][300/388] Elapsed 5m 37s (remain 1m 37s) Loss: 0.0173(0.1412) Grad: 0.9017  LR: 0.00001733  
Epoch: [5][387/388] Elapsed 7m 14s (remain 0m 0s) Loss: 0.5483(0.1380) Grad: 24.0078  LR: 0.00001708  
EVAL: [0/98] Elapsed 0m 0s (remain 0m 49s) Loss: 0.0088(0.0088) 


Epoch 5 - avg_train_loss: 0.1380  avg_val_loss: 0.1599  time: 475s
Epoch 5 - Score: 0.8663
Epoch 5 - Save Best Score: 0.8663 Model


EVAL: [97/98] Elapsed 0m 39s (remain 0m 0s) Loss: 0.0031(0.1599) 


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Epoch: [6][0/388] Elapsed 0m 1s (remain 6m 27s) Loss: 0.1995(0.1995) Grad: 6.5479  LR: 0.00001708  


In [ ]:
sum(oof_df["predictions"]==oof_df["Gender_bin"])/len(oof_df)